#                                  Facial Expression Recognition

Data sets used : https://www.kaggle.com/c/challenges-in-representation-learning-facial-expression-recognition-challenge/

In [1]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df=pd.read_csv('icml_face_data.csv')
df.head()

,emotion,Usage,pixels
0,0,Training,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...
1,0,Training,151 150 147 155 148 133 111 140 170 174 182 15...
2,2,Training,231 212 156 164 174 138 161 173 182 200 106 38...
3,4,Training,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...
4,6,Training,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...


In [3]:
df.Usage.unique()

array(['Training', 'PublicTest', 'PrivateTest'], dtype=object)

In [4]:
df.shape  #Total 35887 data sets

(35887, 3)

In [5]:
df.emotion.unique()

array([0, 2, 4, 6, 3, 5, 1], dtype=int64)

In [6]:
exp_class=['Angry','Disgust','Fear','Happy','Sad','Surprise','Neutral']
exp_class[df.emotion[0]]

'Angry'

In [7]:
df.dtypes

emotion     int64
Usage      object
pixels     object
dtype: object

In [8]:
df1=df.copy()
df1.head()

,emotion,Usage,pixels
0,0,Training,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...
1,0,Training,151 150 147 155 148 133 111 140 170 174 182 15...
2,2,Training,231 212 156 164 174 138 161 173 182 200 106 38...
3,4,Training,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...
4,6,Training,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...


In [9]:
df1.emotion.unique()

array([0, 2, 4, 6, 3, 5, 1], dtype=int64)

In [10]:
df1=pd.get_dummies(df,columns=['emotion'])
df1.head(5)

,Usage,pixels,emotion_0,emotion_1,emotion_2,emotion_3,emotion_4,emotion_5,emotion_6
0,Training,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,1,0,0,0,0,0,0
1,Training,151 150 147 155 148 133 111 140 170 174 182 15...,1,0,0,0,0,0,0
2,Training,231 212 156 164 174 138 161 173 182 200 106 38...,0,0,1,0,0,0,0
3,Training,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,0,0,0,0,1,0,0
4,Training,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,0,0,0,0,0,0,1


In [11]:
df1['pixels'] = df1['pixels'].apply(lambda im: np.fromstring(im, sep=' '))
df1.dtypes

Usage        object
pixels       object
emotion_0     uint8
emotion_1     uint8
emotion_2     uint8
emotion_3     uint8
emotion_4     uint8
emotion_5     uint8
emotion_6     uint8
dtype: object

In [12]:
df1.Usage.value_counts()

Training       28709
PrivateTest     3589
PublicTest      3589
Name: Usage, dtype: int64

In [13]:
train=df1[df1['Usage']=='Training']
test=df1[df1['Usage']=='PrivateTest']
validation=df1[df1['Usage']=='PublicTest']

train.sample(5)

,Usage,pixels,emotion_0,emotion_1,emotion_2,emotion_3,emotion_4,emotion_5,emotion_6
19571,Training,"[21.0, 26.0, 27.0, 28.0, 26.0, 29.0, 27.0, 48....",0,0,0,0,0,0,1
8171,Training,"[29.0, 32.0, 30.0, 32.0, 31.0, 31.0, 33.0, 24....",0,0,0,1,0,0,0
23143,Training,"[255.0, 255.0, 255.0, 255.0, 254.0, 253.0, 255...",0,0,0,0,0,0,1
351,Training,"[26.0, 25.0, 26.0, 27.0, 29.0, 34.0, 37.0, 43....",0,0,1,0,0,0,0
2421,Training,"[182.0, 183.0, 184.0, 185.0, 186.0, 188.0, 190...",0,0,0,1,0,0,0


In [14]:
X_train=np.vstack(train['pixels'].values/255)
X_test=np.vstack(test['pixels'].values/255)
X_valid=np.vstack(validation['pixels'].values/255)
X_train.shape

(28709, 2304)

In [15]:
Y_test=test.drop(['pixels','Usage'],axis='columns')
Y_train=train.drop(['pixels','Usage'],axis='columns')
Y_valid=validation.drop(['pixels','Usage'],axis='columns')

X_train.shape, Y_train.shape,X_test.shape,Y_test.shape, X_valid.shape,Y_valid.shape

((28709, 2304), (28709, 7), (3589, 2304), (3589, 7), (3589, 2304), (3589, 7))

In [16]:
X_test[0]

array([0.66666667, 0.4627451 , 0.39607843, ..., 0.62352941, 0.52156863,
       0.51372549])

In [17]:
Y_test.reset_index(drop=True,inplace=True)
Y_train.reset_index(drop=True,inplace=True)
Y_valid.reset_index(drop=True,inplace=True)

In [18]:
X_train=X_train.reshape(-1,48,48,1)
X_test=X_test.reshape(-1,48,48,1)
X_valid=X_valid.reshape(-1,48,48,1)
X_train.shape

(28709, 48, 48, 1)

# Implementing CNN and ANN

In [19]:
X_train[0].shape

(48, 48, 1)

In [20]:
X_train.shape

(28709, 48, 48, 1)

In [21]:
model=keras.Sequential([
    #CNN part with 4 hidden layers with relu activation and max pooling
    keras.layers.Conv2D(filters=32,kernel_size=(3,3),input_shape=(48,48,1),padding='same',strides=(1, 1),activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D(pool_size=(2,2),strides=(2,2),padding='valid'),
    keras.layers.Dropout(0.25),
    
    keras.layers.Conv2D(filters=64,kernel_size=(3,3),padding='same',strides=(1, 1),activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D(pool_size=(2,2),strides=(2,2),padding='valid'),
    keras.layers.Dropout(0.25),
    
    keras.layers.Conv2D(filters=128,kernel_size=(3,3),padding='same',strides=(1, 1),activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D(pool_size=(2,2),strides=(2,2),padding='valid'),
    keras.layers.Dropout(0.25),
    
    keras.layers.Conv2D(filters=256,kernel_size=(3,3),padding='same',strides=(1, 1),activation='relu'), 
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D(pool_size=(2,2),strides=(2,2),padding='valid'), 
    keras.layers.Dropout(0.25),
    
    
    #ANN part
    keras.layers.Flatten(),
    
    keras.layers.Dense(256,activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.25),
    
    keras.layers.Dense(128,activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.25),
    
    keras.layers.Dense(7,activation='softmax'),

])

model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])


model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 48, 48, 32)        320       
_________________________________________________________________
batch_normalization (BatchNo (None, 48, 48, 32)        128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 24, 24, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 24, 24, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
batch_normalization_1 (Batch (None, 24, 24, 64)        256       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 64)        0

In [22]:
from livelossplot import PlotLossesKeras
from livelossplot.tf_keras import PlotLossesCallback

checkpoint=keras.callbacks.ModelCheckpoint('model_weights.h5',monitor='val_accuracy',save_weights_only=True,mode='max',verbose=1)
reduce_lr=keras.callbacks.ReduceLROnPlateau(monitor='loss',factor=0.1,patience=2,min_lr=0.00001,mode='auto')

callbacks=[checkpoint,reduce_lr]

In [23]:
history=model.fit(X_train,Y_train,epochs=15,batch_size=64,callbacks=callbacks)

Epoch 1/15
449/449 [==============================] - ETA: 0s - loss: 1.9247 - accuracy: 0.2745
Epoch 00001: saving model to model_weights.h5
449/449 [==============================] - 334s 743ms/step - loss: 1.9247 - accuracy: 0.2745
Epoch 2/15
449/449 [==============================] - ETA: 0s - loss: 1.5377 - accuracy: 0.4091
Epoch 00002: saving model to model_weights.h5
449/449 [==============================] - 303s 674ms/step - loss: 1.5377 - accuracy: 0.4091
Epoch 3/15
449/449 [==============================] - ETA: 0s - loss: 1.3635 - accuracy: 0.4779
Epoch 00003: saving model to model_weights.h5
449/449 [==============================] - 294s 654ms/step - loss: 1.3635 - accuracy: 0.4779
Epoch 4/15
449/449 [==============================] - ETA: 0s - loss: 1.2725 - accuracy: 0.5140
Epoch 00004: saving model to model_weights.h5
449/449 [==============================] - 309s 688ms/step - loss: 1.2725 - accuracy: 0.5140
Epoch 5/15
449/449 [==============================] - ETA: 0

In [24]:
model.evaluate(X_test,Y_test)

113/113 [==============================] - 4s 37ms/step - loss: 1.0589 - accuracy: 0.6121


[1.0588597059249878, 0.6121482253074646]

In [25]:
model.evaluate(X_valid,Y_valid)

113/113 [==============================] - 4s 36ms/step - loss: 1.0787 - accuracy: 0.5999


[1.0786622762680054, 0.5998885631561279]

In [26]:
model_json=model.to_json()
with open('model.json','w') as json_file:
    json_file.write(model_json)